## Molkit

In [1]:
# import libraries
import numpy as np
from moha import HamHeisenberg

J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]
                 ])

mu = np.zeros(2)

ham = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham.generate_one_body_integral(dense=True)
two_body = ham.generate_two_body_integral(dense=True)

In [1]:
import numpy as np
from moha import HamHeisenberg
from moha.molkit.hamiltonians import MolHam
from moha.molkit.utils.tools import from_geminal
from moha.molkit.utils.spinops import antisymmetrize_two_body

def from_geminal(two_body_gem, n_orb):
    """
    Inverse of MolHam.to_geminal().

    Parameters
    ----------
    two_body_gem : (n_gem, n_gem) ndarray
        Matrix in the geminal basis.
    n_orb : int
        Number of spin orbitals.

    Returns
    -------
    V : (n_orb, n_orb, n_orb, n_orb) ndarray
        Fully antisymmetrised two-electron tensor V_{ijkl}.
    """
    n_gem = n_orb * (n_orb - 1) // 2
    if two_body_gem.shape != (n_gem, n_gem):
        raise ValueError(f"Shape mismatch: got {two_body_gem.shape}, expected ({n_gem},{n_gem})")

    # Generate flattened pair list exactly like to_geminal
    pairs = [(i, j) for i in range(n_orb) for j in range(i + 1, n_orb)]
    V = np.zeros((n_orb, n_orb, n_orb, n_orb))

    for A, (i, j) in enumerate(pairs):
        for B, (k, l) in enumerate(pairs):
            val = 0.25 * two_body_gem[A, B]  # undo the factor 0.5 * 4 = 2

            # Apply antisymmetric filling
            V[i, j, k, l] = val
            V[j, i, k, l] = -val
            V[i, j, l, k] = -val
            V[j, i, l, k] = val
            V[k, l, i, j] = val
            V[l, k, i, j] = -val
            V[k, l, j, i] = -val
            V[l, k, j, i] = val

    return V


J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]])
mu = np.zeros(2)

ham_hei  = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham_hei.generate_one_body_integral(dense=True)
two_body = ham_hei.generate_two_body_integral(dense=True)
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)

h1_spin, h2_spin = Molecular_Hamiltonian.spinize_H()
h2_spin = antisymmetrize_two_body(h2_spin)

reduced = Molecular_Hamiltonian.to_reduced(n_elec=2)

# Energies Tests

In [ ]:
import pyscf
from pyscf import gto, scf, fci
from pyscf.tools import fcidump
import os


path_xyz = r'../data/hydrogenic/H4/Paldus/h4_0.456_2.xyz'
mol = gto.M(atom=path_xyz, basis='sto-6g')
hf = scf.RHF(mol)

hf.conv_tol = 1.0e-12
status = hf.run().converged
print(status)


mo_energy = hf.mo_energy
mo_occ = hf.mo_occ

print('starting writing')
# Write FCIDUMP

fcidump.from_mo(mol, 'tmp1.fcidump',
                hf.mo_coeff)

# fcidump.from_scf(hf, 
# f"tmp.fcidump",
# tol=0.0)

print('done')

data = fcidump.read("tmp1.fcidump")
e, ci_vec = fci.direct_spin1.kernel(data['H1'],
                                    data['H2'], 
                                    data['NORB'], 
                                    data['NELEC'], ecore=data['ECORE'], nroots=1)

rdm1s, rdm2s = fci.direct_spin1.make_rdm12s(ci_vec, data['NORB'], data['NELEC'])
rdm1 = np.block([[rdm1s[0], np.zeros((4, 4))],
                 [np.zeros((4, 4)), rdm1s[1]]])

rdm2 = np.zeros((8, 8, 8, 8))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                rdm2[i, j, k, l] = rdm2s[0][i, k, j, l]
                rdm2[i+4, j+4, k+4, l+4] = rdm2s[2][i, k, j, l]
                rdm2[i, j+4, k, l+4] = rdm2s[1][i, k, j, l]
                # rdm2[i+4, j, k+4, l] = rdm2s[1][k, i, l, j]
                rdm2[i+4, j, k+4, l] = rdm2s[1][i, k, j, l]
                

# converting rdm2 to physics notation
# rdm2 = np.einsum('ijkl->ikjl', rdm2)
print("trace of rdm1", np.einsum('ii', rdm1))
print("Trace of rdm2", np.einsum('ijij', rdm2))

# loading integrals
# iodata stores integrals in physics notations
# https://iodata.readthedocs.io/en/latest/_modules/iodata/formats/fcidump.html#load_one
integrals = load_one(open('tmp1.fcidump', 'r'))
os.remove('tmp1.fcidump')

h1s = integrals['one_ints']['core_mo']
h2s = integrals['two_ints']['two_mo']

h1 = np.block([[h1s, np.zeros((4, 4))],
               [np.zeros((4, 4)), h1s]])

h2 = np.zeros((8, 8, 8, 8))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # aaaa
                h2[i, j, k, l] = h2s[i, j, k, l]
                # bbbb
                h2[i+4, j+4, k+4, l+4] = h2s[i, j, k, l]
                # abab
                h2[i, j+4, k, l+4] = h2s[i, j, k, l]
                # baba
# h2[i+4, j, k+4, l] = h2s[k, l, i, j]
                h2[i+4, j, k+4, l] = h2s[i, j, k, l]
                
                
res = 0
for i in range(8):
    for j in range(8):
        assert rdm2[i, j, i, j] >=0
        res += rdm2[i, j, i, j]
        
print("trace of 2dm with for loop: ", res)

e_dm = np.einsum('pq, pq', h1, rdm1) + 0.5 * np.einsum('pqrs, pqrs', h2, rdm2)


print(e_dm, e - integrals['core_energy'])


#-----------
eye = np.eye(8)
N = 4
N_alpha = N//2

h1_ups = 0.5 * (np.kron(h1, eye) + np.kron(eye, h1)) / (N-1)
h1_ups = h1_ups.reshape(8, 8, 8, 8)
H = h1_ups + 0.5 * h2

np.einsum('pqrs, pqrs', H, rdm2), e_dm

# to geminal:
# H_gem = to_geminal(H)
# rdm_gem = to_geminal(rdm2)
# np.einsum('pq, pq', H_gem, rdm_gem) == np.einsum('pqrs, pqrs', H, rdm2) == e_dm

In [ ]:
import numpy as np
from moha.molkit.hamiltonians import MolHam     
h_sp = np.array([[1.0, 0.2],
                 [0.2, 1.5]])

V_sp = np.zeros((2, 2, 2, 2))
V_sp[0, 0, 0, 0] = 0.7
V_sp[1, 1, 1, 1] = 0.6

N_ELEC = 2

toy_ham = MolHam(one_body=h_sp, two_body=V_sp)

h_spin, V_spin = toy_ham.spinize_H()     
n_spin = toy_ham.n_spin      

e_fci, ci_vec = fci.direct_spin1.kernel(
    h_spin, V_spin, n_spin, N_ELEC, ecore=0.0, nroots=1
)

rdm1s, rdm2s = fci.direct_spin1.make_rdm12(ci_vec, n_spin, N_ELEC)
rdm1 = np.block([[rdm1s[0], np.zeros((4, 4))],
                 [np.zeros((4, 4)), rdm1s[1]]])

rdm2 = np.zeros((8, 8, 8, 8))
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                rdm2[i, j, k, l] = rdm2s[0][i, k, j, l]
                rdm2[i+4, j+4, k+4, l+4] = rdm2s[2][i, k, j, l]
                rdm2[i, j+4, k, l+4] = rdm2s[1][i, k, j, l]
                # rdm2[i+4, j, k+4, l] = rdm2s[1][k, i, l, j]
                rdm2[i+4, j, k+4, l] = rdm2s[1][i, k, j, l]

In [ ]:
import numpy as np
from pyscf import fci

from moha.molkit.hamiltonians import MolHam
from moha.utils.spinops import (
    to_geminal_rdm2
)


h_sp = np.array([[1.0, 0.2],
                 [0.2, 1.5]])

V_sp = np.zeros((2, 2, 2, 2))
V_sp[0, 0, 0, 0] = 0.7
V_sp[1, 1, 1, 1] = 0.6

N_ELEC = 2                   
toy = MolHam(h_sp, V_sp)

h_spin, V_spin = toy.spinize_H()           

e_fci, ci_vec = fci.direct_spin1.kernel(h_spin, V_spin, n_spin, N_ELEC, ecore=0.0)
rdm1, rdm2 = fci.direct_spin1.make_rdm12(ci_vec, n_spin, N_ELEC)#is this correct?

H = toy.to_reduced(n_elec=N_ELEC)  # H = h↑ + 0.5 * V

E_spin = np.einsum('pqrs,pqrs', H, rdm2)

##teste
H_gem   = toy.to_geminal(H)
rdm_gem = to_geminal_rdm2(rdm2)
E_gem   = np.einsum('pq,pq', H_gem, rdm_gem)

print("E_spin-orbital :", E_spin)
print("E_geminal      :", E_gem)

assert np.allclose(E_spin, E_gem)